# 10: Recommender Systems

The hand-in exercise for this topic is Exercise 1 from the notebook “Exercises in Recommender systems.ipynb"

---

## Exercise 1

Using the "Coursera Courses Dataset 2021" available at kaggle ([https://www.kaggle.com/datasets/khusheekapoor/coursera-courses-dataset-2021](https://www.kaggle.com/datasets/khusheekapoor/coursera-courses-dataset-2021)) or on moodle, to do the following:

1. Create a Content-based filtering recommender system based on the Course Descriptions.
2. Create a Content-based filtering recommender system based on the Skills.

Using the "Book Recommendation Dataset" available at kaggle ([https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset](https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset)) or on moodle, to do the following:

3. Load in the `Ratings.csv` file (on moodle, it is called `Books_Ratings.csv`). Group by `User-ID` and sort by `Book-Rating` in descending order to get the users who rated most books. Filter the rating data to only contain the 200 users that rated most books.
4. Create a Collaborative filtering recommender system based on the user ratings from 3 together with the `Books.csv` dataset.

---

In [34]:
# Lib imports for the notebook
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

---

### Coursera Courses Dataset 2021
1. Create a Content-based filtering recommender system based on the Course Descriptions.
2. Create a Content-based filtering recommender system based on the Skills.

Some EDA to prepare and look at the structure of the data

In [35]:
coursera_data = pd.read_csv('Coursera.csv')

In [36]:
coursera_data.head(10)

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...
5,Building Test Automation Framework using Selen...,Coursera Project Network,Beginner,4.7,https://www.coursera.org/learn/building-test-a...,Selenium is one of the most widely used functi...,maintenance test case test automation scree...
6,Doing Business in China Capstone,The Chinese University of Hong Kong,Advanced,3.3,https://www.coursera.org/learn/doing-business-...,Doing Business in China Capstone enables you t...,marketing plan Planning Marketing consumpti...
7,"Programming Languages, Part A",University of Washington,Intermediate,4.9,https://www.coursera.org/learn/programming-lan...,This course is an introduction to the basic co...,inference ml (programming language) higher-o...
8,The Roles and Responsibilities of Nonprofit Bo...,The State University of New York,Intermediate,4.3,https://www.coursera.org/learn/nonprofit-gov-2,This course provides a more in-depth look at t...,Planning Peer Review fundraising strategic ...
9,Business Russian Communication. Part 3,Saint Petersburg State University,Intermediate,Not Calibrated,https://www.coursera.org/learn/business-russia...,Russian is considered to be one of the most di...,Russian market (economics) tax exemption co...


In [37]:
coursera_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3522 entries, 0 to 3521
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Course Name         3522 non-null   object
 1   University          3522 non-null   object
 2   Difficulty Level    3522 non-null   object
 3   Course Rating       3522 non-null   object
 4   Course URL          3522 non-null   object
 5   Course Description  3522 non-null   object
 6   Skills              3522 non-null   object
dtypes: object(7)
memory usage: 192.7+ KB


---

### 1. Create a Content-based filtering recommender system based on the Course Descriptions.

### Recommendation based on similariry in 'Description' feature

* Write the name of a course from the data you like and the function will recommend courses with similar term frequency

* I like the course "Retrieve Data using Single-Table SQL Queries" so im going to look for 5 similar ones

In [38]:
# Use TFIDF vectorization to calc term frequency and match documents with similar vectors
tfidf_vectorizer = TfidfVectorizer(stop_words='english') # skip common english words so they dont affect the search
# Fit and transform the Course Descriptions
tfidf_matrix = tfidf_vectorizer.fit_transform(coursera_data['Course Description'])
# we need the cosine similarity matrix, to find similarities between input description vs lookup
cosine_similar_descriptions = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Func for recommending similar courses to the search, based on simiar term frequency
def recommend_courses_based_on_desc(course_title, num_recommendations):
    if course_title not in coursera_data['Course Name'].values:
        return "Course not found." # if the course dont exist in the data

    idx = coursera_data[coursera_data['Course Name'] == course_title].index[0]
    similarity_scores = list(enumerate(cosine_similar_descriptions[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:num_recommendations+1]
    recommended_courses = [coursera_data.iloc[i[0]]['Course Name'] for i in similarity_scores]
    return recommended_courses

# Make a search and get a recommendation on the course i like
recommend_courses_based_on_desc("Retrieve Data using Single-Table SQL Queries", 5) # I want 5 recommendationss

['Creating Database Tables with SQL',
 'Querying Databases Using SQL SELECT statement',
 'Advanced Relational Database and SQL',
 'Create Relational Database Tables Using SQLiteStudio',
 'Intermediate Relational Database and SQL']

---

### 2. Create a Content-based filtering recommender system based on the Skills

* We can use the same syntax as before, but instead targeting the 'Skills' feature

* Write the name of a course from the data you like and the function will recommend courses with similar term frequency

* I like the course "Retrieve Data using Single-Table SQL Queries" so im going to look for 5 similar ones

In [39]:
# Use TFIDF vectorization to calc term frequency and match documents with similar vectors
tfidf_vectorizer = TfidfVectorizer(stop_words='english') # skip common english words so they dont affect the search
# Fit and transform the Course Descriptions
tfidf_matrix = tfidf_vectorizer.fit_transform(coursera_data['Skills'])
# we need the cosine similarity matrix, to find similarities between input description vs lookup
cosine_similar_descriptions = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Func for recommending similar courses to the search, based on simiar term frequency
def recommend_courses_based_on_desc(course_title, num_recommendations):
    if course_title not in coursera_data['Course Name'].values:
        return "Course not found." # if the course dont exist in the data

    idx = coursera_data[coursera_data['Course Name'] == course_title].index[0]
    similarity_scores = list(enumerate(cosine_similar_descriptions[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:num_recommendations+1]
    recommended_courses = [coursera_data.iloc[i[0]]['Course Name'] for i in similarity_scores]
    return recommended_courses

# Make a search and get a recommendation on the course i like
recommend_courses_based_on_desc("Retrieve Data using Single-Table SQL Queries", 5) # I want 5 recommendationss as defined as parameter for the func

['Manipulating Data with SQL',
 'Creating Database Tables with SQL',
 'Managing Big Data with MySQL',
 'Create Relational Database Tables Using SQLiteStudio',
 'Retrieve Data with Multiple-Table SQL Queries']

---

### Book Recommendation Dataset
3. Load in the Ratings.csv file (on moodle, it is called Books_Ratings.csv). Group by User-ID and sort by Book-Rating in descending order to get the users who rated most books. Filter the rating data to only contain the 200 users that rated most books.
4. Create a Collaborative filtering recommender system based on the user ratings from 3 together with the Books.csv dataset.

### First steps EDA

In [40]:
books_ratings_data = pd.read_csv('Books_Ratings.csv')

In [41]:
books_ratings_data.head(10)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
5,276733,2080674722,0
6,276736,3257224281,8
7,276737,0600570967,6
8,276744,038550120X,7
9,276745,342310538,10


In [42]:
books_ratings_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


---

### 3. Load in the Books_Ratings.csv file. Group by User-ID and sort by Book-Rating in descending order to get the users who rated most books. Filter the rating data to only contain the 200 users that rated most books.

In [47]:
# Load the book ratings dataset
df_ratings = pd.read_csv("Books_Ratings.csv")
# Count how many books each user has rated
user_counts = df_ratings.groupby("User-ID")["Book-Rating"].count().reset_index()
# Get the top 200 users with the most ratings
top_200_users = user_counts.sort_values(by="Book-Rating", ascending=False).head(200)["User-ID"]
# Filter ratings dataset to only include the top 200 users
filtered_ratings = df_ratings[df_ratings["User-ID"].isin(top_200_users)]

# Load the books dataset
df_books = pd.read_csv("Books.csv")
# Merge ratings with book details using ISBN
df_merged = filtered_ratings.merge(df_books, on="ISBN", how="left")
# Keep only relevant columns 
df_merged = df_merged[["User-ID", "ISBN", "Book-Title", "Book-Rating"]]
# Display dataset
print(df_merged.head())

C:\Users\abjar\AppData\Local\Temp\ipykernel_29172\931618713.py:11: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books = pd.read_csv("Books.csv")


   User-ID        ISBN                                         Book-Title  \
0   278418  0006128831                                                NaN   
1   278418  0006542808                              Silence of the Sirens   
2   278418  0020209606                                NEVER ALONE REISSUE   
3   278418  0020418809                                    CADDIE WOODLAWN   
4   278418  0020420900  Paul Revere : Boston Patriot (Childhood Of Fam...   

   Book-Rating  
0            0  
1            5  
2            0  
3            0  
4            0  


---

### 4. Create a Collaborative filtering recommender system based on the user ratings from 3 together with the Books_Ratings.csv dataset.

In [ ]:
# Create a User-Item rating matrix (Users as rows, Books as columns)
user_item_matrix = df_merged.pivot_table(index="User-ID", columns="Book-Title", values="Book-Rating", fill_value=0)
# Compute cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix)
# Convert similarity matrix into a DataFrame
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)
# Display similarity matrix
print(user_similarity_df.head())


User-ID    3363      6251      6575      7346      11601     11676     12538   \
User-ID                                                                         
3363     1.000000  0.000000  0.026805  0.013205  0.000000  0.022112  0.009240   
6251     0.000000  1.000000  0.048537  0.014077  0.019330  0.037508  0.000000   
6575     0.026805  0.048537  1.000000  0.055537  0.012073  0.063106  0.020387   
7346     0.013205  0.014077  0.055537  1.000000  0.007492  0.051761  0.034727   
11601    0.000000  0.019330  0.012073  0.007492  1.000000  0.020077  0.019004   

User-ID    13552     15408     16634   ...    264321    265115    265313  \
User-ID                                ...                                 
3363     0.007907  0.012508  0.000000  ...  0.005234  0.005454  0.000000   
6251     0.010237  0.006046  0.020697  ...  0.013283  0.007398  0.024124   
6575     0.021962  0.018973  0.010158  ...  0.019694  0.032359  0.012910   
7346     0.025568  0.015265  0.009065  ...  0.015554

In [ ]:
def recommend_books(user_id, rating_matrix, user_similarity_df, df_books, top_n=5):

    if user_id not in user_similarity_df.index:
        return "User not found in top 200!"
    # Get similarity scores for the user, excluding themselves
    sim_scores = user_similarity_df.loc[user_id].drop(user_id)
    # Select top 5 most similar users
    similar_users = sim_scores.sort_values(ascending=False).head(5).index
    # Aggregate ratings from similar users
    similar_ratings = rating_matrix.loc[similar_users]
    # Compute the average rating for each book among these similar users
    avg_ratings = similar_ratings.mean(axis=0)
    # Find books the target user has not rated
    target_user_ratings = rating_matrix.loc[user_id]
    unrated_books = target_user_ratings[target_user_ratings == 0].index
    # Filter ratings to only include unrated books
    recommendations = avg_ratings.loc[unrated_books].sort_values(ascending=False).head(top_n)
    # Convert index (Book-Title) to a list for merging with book details
    recommended_books = df_books[df_books['Book-Title'].isin(recommendations.index)][['ISBN', 'Book-Title', 'Book-Author']]
    
    return recommended_books

In [46]:
example_user = top_200_users.iloc[0]  # Select the first user in the top 200

top_books = recommend_books(example_user, user_item_matrix, user_similarity_df, df_books, top_n=5)

print(f"Top {len(top_books)} recommended books for User {example_user}:")
print(top_books)

Top 10 recommended books for User 11676:
              ISBN                                    Book-Title  \
1360    0671759361                    Pearl in the Mist (Landry)   
4269    0316284955  White Oleander : A Novel (Oprah's Book Club)   
15611   0064400026                   Little House on the Prairie   
24362   0064400069                The Long Winter (Little House)   
26127   0440167531                                      Palomino   
43124   0060581859                The Long Winter (Little House)   
108090  0060522410                The Long Winter (Little House)   
188054  0702219487                                      Palomino   
202942  0061070068                   Little House on the Prairie   
219583  044056753X                                      Palomino   

                 Book-Author  
1360            V.C. Andrews  
4269             Janet Fitch  
15611   Laura Ingalls Wilder  
24362   Laura Ingalls Wilder  
26127         DANIELLE STEEL  
43124   Laura Ingalls Wi

---